In [333]:
import os
import datetime
from collections import Counter
import pandas as pd
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [159]:
% cd /Users/mdymshits/aviv/data
% ls

/Users/mdymshits/aviv/data
drive-download-20170512T141423Z-001/
drive-download-20170512T141423Z-001.zip
mixed/
results-20170419-short-esp-type-1.xlsx*
results-20170420-long-esp-type-2-18.csv
results-20170420-long-esp-type-2-18.xlsx*
results-20170420-long-esp-type-2.xlsx*
results-20170421-capucio-type2-24.xlsx
results-20170501-latte-type2-10.xlsx


### Initial parameters

In [224]:
short_esp_threshold = 0.8
time_block_value = 85

### Main functions

In [344]:
def reading_file(path):
    if path.endswith('xlsx'):
        df = pd.read_excel(path)
    elif path.endswith('csv'):
        df = pd.read_csv(path, names=['date', 'ActivePower'],  header=0)
        df['date'] = pd.to_datetime(df['date'])
    return df

def caffe_blocks_extraction(df):
    blocks = []
    next_line = 0
    for i in range(df.shape[0]):
        line = df.iloc[next_line]
        if line['ActivePower'] > short_esp_threshold:
            start = line['date']
#             print next_line
            end = line['date'] + datetime.timedelta(0,time_block_value)
            block = df[(df['date'] > start) & (df['date'] <= end)]
            blocks.append(block )
            next_line += block.shape[0]
        else:
            next_line += 1
        if next_line > df.shape[0] - 1:
            break
    return blocks

def feature_extraction(blocks):
    return [b.describe().values.flatten().tolist() for b in blocks]
#     return [[b.std().values[0], b.mean().values[0]] for b in blocks]
    
def path_to_features(path):
    df = reading_file(path)
    blocks = caffe_blocks_extraction(df)
    print 'there is about {} cups'.format(len(blocks))
    return feature_extraction(blocks)

### Feature extractions, and labels creation

In [345]:
path_l = '/Users/mdymshits/aviv/data/results-20170501-latte-type2-10.xlsx'
path_e = '/Users/mdymshits/aviv/data/results-20170420-long-esp-type-2-18.csv'
path_c = '/Users/mdymshits/aviv/data/results-20170421-capucio-type2-24.xlsx'
path_es = '/Users/mdymshits/aviv/data/results-20170419-short-esp-type-1.xlsx'

features_l, labels_l = path_to_features(path_l), [1] * 10
features_e, labels_e = path_to_features(path_e), [0] * 18
features_c, labels_c = path_to_features(path_c), [1] * 29
features_es, labels_es = path_to_features(path_es), [0] * 26
features = features_l + features_e + features_c + features_es
labels = labels_l + labels_e + labels_c + labels_es

there is about 10 cups
there is about 18 cups
there is about 29 cups
there is about 26 cups


### Training Classifier

In [346]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)
# clf = LinearSVC(C=1)
# clf = SVC(kernel='rbf')
clf = LogisticRegression()

[[18  1]
 [ 2 21]]
0.928571428571


### Performance evaluation
If the meaning of confusion matrix is not clear, look at [Confusion Matrix page](https://en.wikipedia.org/wiki/Confusion_matrix)

In [361]:
print 'confusiont matrix \n {} \n'.format(confusion_matrix(
        y_test, preds))
print 'accuracy = {0:0.3f}'.format(accuracy_score(y_test, preds))

confusiont matrix 
 [[18  1]
 [ 2 21]] 

accuracy = 0.929
